<a href="https://colab.research.google.com/github/Mikaral/projetoCripto/blob/main/digitalCash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [426]:
pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [427]:
import rsa
import random
from binascii import hexlify, unhexlify
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from Crypto.Protocol.SecretSharing import Shamir
from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto import Random

In [428]:
def generate_keys():
    (pubKey, privKey) = rsa.newkeys(1024)
    with open('keys/pubkey.pem', 'wb') as f:
        f.write(pubKey.save_pkcs1('PEM'))
        f.close()

    with open('keys/privkey.pem', 'wb') as f:
        f.write(privKey.save_pkcs1('PEM'))
        f.close()

In [429]:
def load_keys():
    with open('keys/pubkey.pem', 'rb') as f:
        pubKey = rsa.PublicKey.load_pkcs1(f.read())

    with open('keys/privkey.pem', 'rb') as f:
        privKey = rsa.PrivateKey.load_pkcs1(f.read())

    return pubKey, privKey

In [430]:
def sign_sha1(msg, key):
    return rsa.sign(msg.encode('ascii'), key, 'SHA-1')

In [431]:
def verify_sha1(msg, signature, key):
    try:
        return rsa.verify(msg.encode('ascii'), signature, key) == 'SHA-1'
    except:
        return False

In [432]:
bancoDados = {}

In [433]:
def criarOrdemPagamento(hash, stringIdentidade, preco):
  return [hash, stringIdentidade, preco, '', '']

In [434]:
alice = [10000, 1234, "8/23"]
bob = [20000, 9876, "8/25"]

In [435]:
def getHash():
    return SHA256.new(data=b'teste')

In [436]:
ordem1 = criarOrdemPagamento(b'123','Alice',150)
ordem2 = criarOrdemPagamento(b'123','bob',100)

In [437]:
def verificarAutenticidadeOrdem(ordem1, ordem2):
  return ordem1[0] == ordem2[0]

In [438]:
assinaturaOficial = "MikaelSilas"

In [439]:
assinaturaFalsa = "Ewerton"

In [440]:
def assinarPacote(ordem1, ordem2, privKey):
  if(verificarAutenticidadeOrdem(ordem1, ordem2)):
    signature = sign_sha1(assinaturaOficial, privKey)
    ordem1[3] = signature
    bancoDados[ordem1[1]] = 0
  else:
    print("As ordens pedem um valor diferente")

In [441]:
def autenticarPacote(ordem, assinatura, pubKey):
   if(verify_sha1(assinatura, ordem[3], pubKey)):
     print("O pacote é autentico")
   else:
     print("O pacote não tem uma assinatura genuíuna")

In [442]:
def gastarDinheiro(ordem, stringId):
  if (bancoDados[ordem[1]] == 0):
    bob[0] += ordem[2]
    bancoDados[ordem[1]] = 1
    ordem[4] = stringId
    bancoDados[ordem[4]] = 1
    alice[0] -= ordem1[2]
    print("Pagamento concluído com Sucesso!")
  else:
    print("O pacote já foi gasto anteriormente") 
    if(ordem[4] in bancoDados.keys()):
      print("Bob está agindo por má fé")
    else:
      print("Alice está agindo por má fé")

##### Caso 01: Transferência ideal de Alice para Bob

##### Caso 02: Alice ou Bob falsificaram a assinatura do banco

In [443]:
resposta = -1

while(resposta != 0):
  print('Cliente: Alice. ${}\nComerciante: Bob. ${}\n'.format(alice[0], bob[0]))
  print('0) Fechar Programa\n1) Realizar transferência\n2) Induzir erro')

  resposta = int(input('>'))
  if(resposta == 1):
    # Criar a ordem de pagamento
    print('Criando a ordem de pagamento para Alice ...')
    ordem1 = criarOrdemPagamento(b'000', 'Alice', 150)
    ordem2 = criarOrdemPagamento(b'000', 'Alice', 100)
    
    # Gerando e importando chaves
    generate_keys()
    pubKey, privKey = load_keys()
    
    # Assinando e autenticando a ordem
    assinarPacote(ordem1, ordem2, privKey)
    autenticarPacote(ordem1, assinaturaOficial, pubKey)

    # Gastando dinheiro
    gastarDinheiro(ordem1, random.randint(0, 100000))
  if(resposta == 2):
    print("1) Alice ou Bob falsificaram a assinatura do banco")
    print("2) Bob tenta gastar o mesmo pacote mais de uma vez")
    print("3) Alice tenta gastar o mesmo pacote mais de uma vez")
    print("4) Alice tenta mandar ordens de diferentes quantias")
    opcao = int(input('> '))
    if(opcao == 1):
      # Criar a ordem de pagamento
      print('Criando a ordem de pagamento para Alice ...')
      ordem1 = criarOrdemPagamento(b'000', 'Alice', 150)
      ordem2 = criarOrdemPagamento(b'000', 'Alice', 100)
      
      # Gerando e importando chaves
      generate_keys()
      pubKey, privKey = load_keys()
      
      # Assinando e autenticando a ordem
      assinarPacote(ordem1, ordem2, privKey)
      autenticarPacote(ordem1, assinaturaFalsa, pubKey)
    if(opcao == 2):
      # Criar a ordem de pagamento
      print('Criando a ordem de pagamento para Alice ...')
      ordem1 = criarOrdemPagamento(b'000', 'Alice', 150)
      ordem2 = criarOrdemPagamento(b'000', 'Alice', 100)
      
      # Gerando e importando chaves
      generate_keys()
      pubKey, privKey = load_keys()
      
      # Assinando e autenticando a ordem
      assinarPacote(ordem1, ordem2, privKey)
      autenticarPacote(ordem1, assinaturaOficial, pubKey)

      # Alice tenta usar o mesmo pagamento duas vezes
      stringId = random.randint(0, 100000)
      ordem1[4] = stringId
      bancoDados[stringId] = 1
      bancoDados[ordem1[1]] = 1
      gastarDinheiro(ordem1, stringId)
    if(opcao == 3):
      # Criar a ordem de pagamento
      print('Criando a ordem de pagamento para Alice ...')
      ordem1 = criarOrdemPagamento(b'000', 'Alice', 150)
      ordem2 = criarOrdemPagamento(b'000', 'Alice', 100)
      
      # Gerando e importando chaves
      generate_keys()
      pubKey, privKey = load_keys()
      
      # Assinando e autenticando a ordem
      assinarPacote(ordem1, ordem2, privKey)
      autenticarPacote(ordem1, assinaturaOficial, pubKey)

      # Alice tenta usar o mesmo pagamento duas vezes
      stringId = random.randint(0, 100000)
      bancoDados[ordem1[1]] = 1
      gastarDinheiro(ordem1, stringId)
    if(opcao == 4):
      # Criar a ordem de pagamento
      print('Criando a ordem de pagamento para Alice ...')
      ordem1 = criarOrdemPagamento(b'001', 'Alice', 150)
      ordem2 = criarOrdemPagamento(b'000', 'Alice', 100)
      
      # Gerando e importando chaves
      generate_keys()
      pubKey, privKey = load_keys()
      
      # Assinando e autenticando a ordem
      assinarPacote(ordem1, ordem2, privKey)

Cliente: Alice. $10000
Comerciante: Bob. $20000

0) Fechar Programa
1) Realizar transferência
2) Induzir erro
>1
Criando a ordem de pagamento para Alice ...
O pacote é autentico
Pagamento concluído com Sucesso!
Cliente: Alice. $9850
Comerciante: Bob. $20150

0) Fechar Programa
1) Realizar transferência
2) Induzir erro
>2
1) Alice ou Bob falsificaram a assinatura do banco
2) Bob tenta gastar o mesmo pacote mais de uma vez
3) Alice tenta gastar o mesmo pacote mais de uma vez
4) Alice tenta mandar ordens de diferentes quantias
> 1
Criando a ordem de pagamento para Alice ...
O pacote não tem uma assinatura genuíuna
Cliente: Alice. $9850
Comerciante: Bob. $20150

0) Fechar Programa
1) Realizar transferência
2) Induzir erro
>2
1) Alice ou Bob falsificaram a assinatura do banco
2) Bob tenta gastar o mesmo pacote mais de uma vez
3) Alice tenta gastar o mesmo pacote mais de uma vez
4) Alice tenta mandar ordens de diferentes quantias
> 3
Criando a ordem de pagamento para Alice ...
O pacote é aut

1) Transferir dinheiro
2) Induzir erro

1) Alice enviou pacotes com diferentes preços
2) Alice ou Bob falsificaram a assinatura do banco
3) Alice tenta gastar a mesma ordem mais de uma vez [ok]
4) Bob tenta gastar a mesma ordem mais de uma vez [ok]

Alice envia 2 pacotes para o banco, cada pacote tem a stringIdentidade1
Antes de abrir, o banco vai calcular o hash dos dois pacotes.
Caso o hash seja igual -> O banco abre um dos pacotes, e desconta da conta de Alice
O segundo pacote vai ser assinado digitalmente (https://pycryptodome.readthedocs.io/en/latest/src/signature/pkcs1_pss.html)
Alice pode gastar o dinheiro em um comerciante.
Na hora de Alice gastar o dinheiro, é gerado uma nova string de identificação, de tal forma que seja diferente da primeira.
Na hora do pagamento, o pacote é enviado pro banco, que autentica com sua chave privada o pagamento.
Caso já exista no banco de dados uma compra com a stringIdentidade1, então o pagamento é recusado e é utilizada a stringIdentidade2 para saber qual das partes é culposa.